In [1]:
from cached_property import cached_property
import cf_units
import datetime
import iris
from iris.analysis.calculus import differentiate
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
#
import umtools.irismode as umu
from umtools.utils import grdstep

In [2]:
import pyveccalc.iris_api as icalc

In [3]:
%matplotlib notebook

In [4]:
# plt.style.reload_library()
plt.style.use('awesome')

In [5]:
iris.FUTURE.netcdf_promote = True
um_res = grdstep('km2p2')

In [6]:
fcst_init = '25_1200'

In [7]:
f = iris.load('/local/fwv14jru/UEA/ACCACIA/UM/exp_results/km2p2/25_1200/model_levels_full/processed/umnsa_rlev_obs_11-14h_014-proc.nc')

In [8]:
f

[<iris 'Cube' of geopotential_height / (m) (model_level_number: 40; grid_latitude: 599; grid_longitude: 599)>,
<iris 'Cube' of surface_altitude / (m) (grid_latitude: 599; grid_longitude: 599)>,
<iris 'Cube' of surface_altitude / (m) (grid_latitude: 599; grid_longitude: 599)>,
<iris 'Cube' of surface_altitude / (m) (grid_latitude: 599; grid_longitude: 599)>,
<iris 'Cube' of surface_altitude / (m) (grid_latitude: 599; grid_longitude: 599)>,
<iris 'Cube' of upward_air_velocity / (m s-1) (model_level_number: 40; grid_latitude: 599; grid_longitude: 599)>,
<iris 'Cube' of x_wind / (m s-1) (model_level_number: 40; grid_latitude: 599; grid_longitude: 599)>,
<iris 'Cube' of y_wind / (m s-1) (model_level_number: 40; grid_latitude: 599; grid_longitude: 599)>]

In [9]:
u_orig = f.extract_strict('x_wind')
v_orig = f.extract_strict('y_wind')
w_orig = f.extract_strict('upward_air_velocity')

In [10]:
u = icalc.cube_ellips_to_cart(u_orig, dx=um_res.to_flt())
v = icalc.cube_ellips_to_cart(v_orig, dx=um_res.to_flt())
w = icalc.cube_ellips_to_cart(w_orig, dx=um_res.to_flt())
w.rename('z_wind')

In [11]:
a = icalc.AtmosFlow(u,v,None)

<iris 'Cube' of shearing_deformation_2d / (hertz) (atmosphere_hybrid_height_coordinate: 40; latitude: 599; longitude: 599)>

In [15]:
import pickle

In [39]:
with open('sizeof_a.pkl', 'wb') as output:
    pickle.dump(a, output, pickle.HIGHEST_PROTOCOL)

In [11]:
class MyClass:
    def __init__(self, x, y):
        self.x = x.copy()
        self.y = y.copy()

    @cached_property
    def wspd(self):
        print("calculating wind speed")
        return self.x * self.y

In [12]:
aa = MyClass(u,v)

In [15]:
aaa = aa.wspd

In [135]:
property??

In [23]:
aa = pickle.load(open('sizeof_u.pkl', 'rb'))

In [28]:
aa.data.nbytes/1024**2

109.49737548828125

In [ ]:
a.rel_vort_tilt

In [ ]:
c_red = mpl.colors.colorConverter.to_rgba('red')
c_blue= mpl.colors.colorConverter.to_rgba('blue')
c_white_trans = mpl.colors.colorConverter.to_rgba('white',alpha = 0.0)
cmap_rb = mpl.colors.LinearSegmentedColormap.from_list('cmap_BuRd',[c_blue,c_white_trans,c_red],64)
cmap_rb.set_over('magenta')
cmap_rb.set_under('navy')

bounds = [-10,-1,1,10]
norm = mpl.colors.BoundaryNorm(bounds, cmap_rb.N)

kw = dict(norm=norm, levels=bounds, cmap=cmap_rb, extend='both')

In [ ]:
plt.contourf(a.rel_vort[8,...].data*1e4, levels=np.linspace(-10,10,11), cmap='PuOr_r', extend='both')
plt.contourf(a.kvn[8,...].data, **kw)
plt.colorbar()

In [ ]:
def _show_points(cube, i1=1, i2=4):
    print(cube.name())
    print(cube.coord(axis='x').name(), cube.coord(axis='x').points[i1:i2], len(cube.coord(axis='x').points))
    print(cube.data[0,0,i1:i2])
    print(cube.coord(axis='y').name(), cube.coord(axis='y').points[i1:i2], len(cube.coord(axis='y').points))
    print(cube.data[0,i1:i2,0])

In [ ]:
_show_points(u)